<a href="https://colab.research.google.com/github/rsrini7/Colabs/blob/main/PydanticAI_Grog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an Agentic System to enhance RAG with Self-Grading and Web Search Capabilities using Pydantic.AI, Groq and Langchain

Reference: [Medium Article](https://medium.com/the-ai-forum/building-an-agentic-system-to-enhance-rag-with-self-grading-and-web-search-capabilities-using-3f9a1d885730)

In [ ]:
!pip install chromadb pydantic-ai nest_asyncio devtools 'pydantic-ai-slim[openai,groq,logfire]' tavily-python langchain langchain_community sentence_transformers langchain_huggingface pypdf -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 56.4 MB/s eta 0:00:

In [18]:
from dataclasses import dataclass
@dataclass
class Deps:
    question:str |None
    context:str |None
    response:str |None

from google.colab import userdata
import os
from pydantic_ai import Agent, RunContext, Tool
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.groq import GroqModel
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from typing import List

os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

openrouter_api_key = userdata.get('OPENROUTER_API_KEY')
openai_model = OpenAIModel('openai/gpt-3.5-turbo',
    provider=OpenAIProvider(api_key=openrouter_api_key,
                    base_url="https://openrouter.ai/api/v1",))
groq_model = GroqModel("llama-3.3-70b-versatile")

persist_directory = "./chroma_data"

loader = PyPDFLoader("./data/Fibromyalgia_Final.pdf")
documents = loader.load()

split_docs = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(documents)

#embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#Build Index
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory,
    collection_name="fibromyalgia"
)

import nest_asyncio
nest_asyncio.apply()
#
groq_agent = Agent(groq_model,
                   deps_type=Deps,
                    retries=2,
                    result_type=str,
                   system_prompt=("You are a Helpful Assiatnt Profiocient in Answering concise,factful and to the point asnwers for questions asked based on the Context provided"
                   "You have to Use the `retriever_tool' to get relevent context and generate response based on the context retrieved"
                   """You are a grading assistant. Evaluate the response based on:
        1. Relevancy to the question
        2. Faithfulness to the context
        3. Context quality and completeness

        lease grade the following response based on:
        1. Relevancy (0-1): How well does it answer the question?
        2. Faithfulness (0-1): How well does it stick to the provided context?
        3. Context Quality (0-1): How complete and relevant is the provided context?

        Question: {ctx.deps.query}
        Context: {ctx.deps.context}
        Response: {ctx.deps.response}

        Also determine if web search is needed to augment the context.

        Provide the grades and explanation in the JSON format with key atrributes 'Relevancy','Faithfulness','Context Quality','Needs Web Search':
        {"Relevancy": <score>,
        "Faithfulness": <score>,
        "Context Quality": <score>,
        "Needs Web Search": <true/false>,
        "Explanation": <explanation>,
        "Answer":<provide response based on the context from the `retriever_tool' if 'Need Web Search' value is 'false' otherwise Use the `websearch_tool` function to generate the final reaponse}"""
        ),
        )

@groq_agent.tool_plain
async def websearch_tool(question) -> str:
    """check if the square is a winner"""
    from tavily import TavilyClient
    # Step 1. Instantiating your TavilyClient
    tavily_client = TavilyClient(userdata.get('TAVILY_API_KEY'))

    # Step 2. Executing a Q&A search query
    answer = tavily_client.qna_search(query=question)

    # Step 3. That's it! Your question has been answered!
    print(f"WEB SEARCH:{answer}")
    return answer

@groq_agent.tool
async def retriever_tool( ctx: RunContext[Deps],question:str)-> List[str]:
  load_vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding,collection_name="fibromyalgia")
  docs = load_vectorstore.similarity_search(question,k=3)
  documnets = [d.page_content for d in docs]
  print(f"RAG Retrieval:{documnets}")
  return documnets

query = "What is Fibromyalgia?"
response = groq_agent.run_sync(query)
print("AgentRunResult: ")
print(response)
print("Response Output: "+response.output)
print("Response Usage: ")
print(response.usage())
print("--------------")

query = "What is Fibromyalgia and what are it's causes?"
response = groq_agent.run_sync(query)
print("AgentRunResult: ")
print(response)
from langchain_core.output_parsers import JsonOutputParser
print("Response Output: "+response.output)
parser = JsonOutputParser()
print("Json Parser: ")
print(parser.parse(response.output))
print(parser.parse(response.output)['Answer'])
print("Response Usage: ")
print(response.usage())
print("--------------")

query = "What is the life expectancy of people suffering with fibromyalgia?"
response = groq_agent.run_sync(query)
print("AgentRunResult: ")
print(response)
print("Response Output: "+response.output)
parser = JsonOutputParser()
print("Json Parser: ")
print(parser.parse(response.output))
print(parser.parse(response.output)['Answer'])
print("Response Usage: ")
print(response.usage())

RAG Retrieval:['etiology of fibromyalgia remains unknown, but recent advances \nand discoveries have helped to unravel some of the mysteries \nof this disease. Research highlights some of the biochemical, \nmetabolic, and immunoregulatory abnormalities associated with \nfibromyalgia. Management of FMS at the present time is very \ndifficult as it has multiple etiological factors and psychological \npredispositions; however, a patient centered approach is essential \nto handle this problem.', 'etiology of fibromyalgia remains unknown, but recent advances \nand discoveries have helped to unravel some of the mysteries \nof this disease. Research highlights some of the biochemical, \nmetabolic, and immunoregulatory abnormalities associated with \nfibromyalgia. Management of FMS at the present time is very \ndifficult as it has multiple etiological factors and psychological \npredispositions; however, a patient centered approach is essential \nto handle this problem.', 'etiology of fibromya